In [ ]:
import numpy as np
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.misc import derivative
import MDAnalysis as mda
from itertools import chain
import pandas as pd

def find_closest_atom(target_atoms, specific_atom):
    distances = np.linalg.norm(target_atoms.positions - specific_atom, axis=1)
    min_distance_index = np.argmin(distances)
    closest_atom_id = target_atoms[min_distance_index].id
    return closest_atom_id, distances[min_distance_index]

def cal_dp_index(u, frame_ranges):
    dp_ndx = []
    for ts in frame_ranges:
        u.trajectory[ts]
        # 选择特定原子组
        atom_indices = [240,241,242,243,245,246,247]
        group1 = u.atoms[atom_indices]
        center_of_mass = group1.center_of_mass()
        specific_atom = center_of_mass
        # 选择ssDNA DB原子组
        target_atoms = u.select_atoms("bynum 991:1283 and name DB")
        # 找到最接近的原子序号
        closest_atom_id, min_distance = find_closest_atom(target_atoms, specific_atom)
        #print(f"closest_atom_id: {closest_atom_id}, min_distance: {min_distance}")
        dp_ndx.append(closest_atom_id)
    return dp_ndx

def find_continuous_segments(filename):
    """load the data"""
    df = pd.read_csv(filename, delim_whitespace=True, header=None)
    """ create'condition',if A > 0.8 and B > 0.8 is True,otherwise, is False """
    df['condition'] = (df.iloc[:, 1] > 0.8) & (df.iloc[:, 2] > 0.8)   
    """ 通过比较当前行的'condition'与上一行的'condition'是否相同，来标识不同的段落
     'cumsum'用于为每个新的段落分配一个新的组号 """
    df['group'] = (df['condition'] != df['condition'].shift()).cumsum()
    """仅选择满足条件的行，并按照组号聚合，获取每个组的第一个和最后一个'index'"""
    filtered_df = df[df['condition']].groupby('group')[0].agg(['first', 'last'])   
    """将每一对'first'和'last'转换为range格式的字符串"""
    ranges = [f'range({row["first"]}, {row["last"] + 1})' for index, row in filtered_df.iterrows()]   
    return ranges

def calculate_centroid(coordinates):
    """计算给定坐标的质心"""
    return np.mean(coordinates, axis=0)

def calculate_angle(v1, v2):
    """计算两个向量之间的夹角（以度为单位）"""
    unit_v1 = v1 / np.linalg.norm(v1)
    unit_v2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_v1, unit_v2)
    angle_rad = np.arccos(np.clip(dot_product, -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)
    return angle_deg

def calculate_min_angle(coordinates):
    """计算并返回最大的角度"""
    min_angle = 180
    n = len(coordinates)
    
    for i in range(3, n-3):
        centroid_prev = calculate_centroid(coordinates[i-3:i])
        centroid_next = calculate_centroid(coordinates[i+1:i+4])
        
        '''当前原子到两个质心的向量'''
        v1 = centroid_prev - coordinates[i]
        v2 = centroid_next - coordinates[i]
        
        '''计算角度'''
        angle = calculate_angle(v1, v2)
        min_angle = min(min_angle, angle)
    
    return min_angle

In [ ]:
min_angle_results0 = []

for i in range(1, 2):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/03_simulation_DNAout_script_delmiss_para0.3-0.5/noDNAProGo_replica/noDNAProGo_0.35/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/03_simulation_DNAout_script_delmiss_para0.3-0.5/noDNAProGo_replica/noDNAProGo_0.35/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/03_simulation_DNAout_script_delmiss_para0.3-0.5/noDNAProGo_replica/analysis/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results0.append(min_angle)

np.savetxt('min_angle_results0', min_angle_results0, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results0, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results0, bins=bins, density=True, alpha=0.75)
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results0))

In [ ]:
min_angle_results1 = []

for i in range(1, 11):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.3/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.3/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific0.3/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results1.append(min_angle)
        
np.savetxt('min_angle_results1', min_angle_results1, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results1, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results1, bins=bins, density=True, alpha=0.75)
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results1))

In [ ]:
min_angle_results2 = []

for i in range(1, 11):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.5/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.5/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific0.5/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results2.append(min_angle)

np.savetxt('min_angle_results2', min_angle_results2, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results2, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results2, bins=bins, density=True, alpha=0.75)
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results2))

In [ ]:
min_angle_results3 = []

for i in range(11, 16):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.6/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.6/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific0.6/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results3.append(min_angle)

np.savetxt('min_angle_results3', min_angle_results3, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results3, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results3, bins=bins, density=True, alpha=0.75, color='purple')
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results3))

In [ ]:
min_angle_results4 = []

for i in range(1, 11):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.7/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.7/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific0.7/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results4.append(min_angle)

np.savetxt('min_angle_results4', min_angle_results4, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results4, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results4, bins=bins, density=True, alpha=0.75, color='purple')
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results4))

In [ ]:
min_angle_results5 = []

for i in range(1, 11):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.9/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_0.9/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific0.9/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results5.append(min_angle)

np.savetxt('min_angle_results5', min_angle_results5, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results5, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results5, bins=bins, density=True, alpha=0.75, color='yellow')
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results5))

In [ ]:
min_angle_results6 = []

for i in range(1, 10):
    u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_1/0.35-{i}/run1/pro_dna_test_run.pdb', f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/para_1/0.35-{i}/run1/pro_dna_test_run.dcd')

    filename = f"/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/analysis/para_nonspecific1/All_knetics/order_parameter/0.35/COLVAR{i}"

    continuous_segments = find_continuous_segments(filename)
    # transform the string to range object
    ranges = [eval(r) for r in continuous_segments]
    #print(ranges)
    # using chain to combine the ranges
    frame_ranges = chain(*ranges)
    dp_ndx = cal_dp_index(u1, frame_ranges)
    converted_list1 = [f"bynum {max(x-65, 991)}:{min(x+63, 1283)} and name DB" for x in dp_ndx]
    converted_list2 = [f"bynum {max(2568-x-65, 1284)}:{min(2568-x+63, 1576)} and name DB" for x in dp_ndx]
    # we should get a new frame_ranges, when we use frame_ranges in dp_index, it will be empty
    frame_ranges = chain(*ranges)

    #print(converted_list)
    m = 0
    for ts in frame_ranges:
        u1.trajectory[ts]
        atoms1 = u1.select_atoms(converted_list1[m])
        atoms2 = u1.select_atoms(converted_list2[m])

        # 确保两组原子数量相等
        if len(atoms1) != len(atoms2):
            raise ValueError("atoms1 和 atoms2 的原子数量不相等")

        # 计算质心并保存
        centroids = []
        for i in range(len(atoms1)):
            atom1 = atoms1[i]
            atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
            centroid = (atom1.position + atom2.position) / 2
            centroids.append(centroid)

        min_angle = calculate_min_angle(centroids)
        min_angle_results6.append(min_angle)

np.savetxt('min_angle_results6', min_angle_results6, delimiter='  ', fmt=['%.2f'], header="#angle", comments='')

In [ ]:
bins = np.arange(0, 180, 1) 

hist, bin_edges = np.histogram(min_angle_results6, bins= bins)
# find the bin with the maximum value
max_bin_index = np.argmax(hist)
# find the value of the bin with the maximum value
max_bin_value = hist[max_bin_index]

print(f"max_bin_value:{max_bin_value}")
print(f"max_bin_index:{max_bin_index}")

# plot probability distribution
plt.hist(min_angle_results6, bins=bins, density=True, alpha=0.75, color='red')
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.show()
print(np.mean(min_angle_results6))

In [ ]:
u1 = mda.Universe(f'/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Full_removLig+98bp/021_delPartGo/021_nonspecific_DNAout_0.35/7q2z.pdb')

atoms1 = u1.select_atoms("bynum 7318:7777 and name N3")
atoms2 = u1.select_atoms("bynum 7779:8261 and name N1")

# 确保两组原子数量相等
if len(atoms1) != len(atoms2):
    raise ValueError("atoms1 和 atoms2 的原子数量不相等")

# 计算质心并保存
centroids = []
for i in range(len(atoms1)):
    atom1 = atoms1[i]
    atom2 = atoms2[-(i+1)]  # 从atoms2的末尾开始选择原子
    centroid = (atom1.position + atom2.position) / 2
    centroids.append(centroid)

min_angle = calculate_min_angle(centroids)
print(f"min_angle: {min_angle}")


In [ ]:
# plot probability distribution
plt.figure(figsize=(15, 6))
plt.hist(min_angle_results0, bins=bins, density=True, alpha=0.75, label='para=0', color='blue')
plt.axvline(np.mean(min_angle_results0), color='blue', linestyle='dashed', linewidth=3, label='para=0')
plt.hist(min_angle_results1, bins=bins, density=True, alpha=0.75, label='para=0.3', color='orange')
plt.axvline(np.mean(min_angle_results1), color='orange', linestyle='dashed', linewidth=3, label='para=0.3')
plt.hist(min_angle_results2, bins=bins, density=True, alpha=0.75, label='para=0.5', color='white')
plt.axvline(np.mean(min_angle_results2), color='white', linestyle='dashed', linewidth=3, label='para=0.5')
plt.hist(min_angle_results3, bins=bins, density=True, alpha=0.75, label='para=0.6', color='green')
plt.axvline(np.mean(min_angle_results3), color='green', linestyle='dashed', linewidth=3, label='para=0.6')
plt.hist(min_angle_results4, bins=bins, density=True, alpha=0.75, label='para=0.7', color='purple')
plt.axvline(np.mean(min_angle_results4), color='purple', linestyle='dashed', linewidth=3, label='para=0.7')
plt.hist(min_angle_results5, bins=bins, density=True, alpha=0.75, label='para=0.9', color='yellow')
plt.axvline(np.mean(min_angle_results5), color='yellow', linestyle='dashed', linewidth=3, label='para=0.9')
plt.hist(min_angle_results6, bins=bins, density=True, alpha=0.75, label='para=1', color='red')
plt.axvline(np.mean(min_angle_results6), color='red', linestyle='dashed', linewidth=3, label='para=1')
# 画一条垂直线, x=144.97105130015322
plt.axvline(144.97105130015322, color='black', linestyle='dashed', linewidth=3, label='PDB')
plt.xlim(0, 180)
plt.xlabel('Min Angle (degrees)')
plt.ylabel('Probability Density')
plt.title('Probability Distribution of Min Angle')
plt.legend()
plt.show()

In [ ]:
# 导入数据
min_angle_results0 = np.loadtxt('min_angle_results0', comments='#')
min_angle_results1 = np.loadtxt('min_angle_results1', comments='#')
min_angle_results2 = np.loadtxt('min_angle_results2', comments='#')
min_angle_results3 = np.loadtxt('min_angle_results3', comments='#')
min_angle_results4 = np.loadtxt('min_angle_results4', comments='#')
min_angle_results5 = np.loadtxt('min_angle_results5', comments='#')
min_angle_results6 = np.loadtxt('min_angle_results6', comments='#')

# 假设你已经加载了你的数据到 min_angle_results0 到 min_angle_results6 变量中

def calculate_SE(data, groups=10):
    """
    将数据分成指定数量的组，计算每个组的平均值，然后计算这些平均值的标准误差(SE)。
    """
    n = len(data)
    # 确保能够平均分组
    if n < groups:
        raise ValueError("数据量少于分组数量，无法分组。")
    
    # 计算每组的大小，使用整除 // 来确保得到整数
    group_size = n // groups
    
    # 初始化一个列表来存储每个子组的平均值
    group_means = []
    
    for i in range(groups):
        # 计算每个子组的开始和结束索引
        start_idx = i * group_size
        # 对于最后一个子组，确保包含所有剩余的数据
        if i == groups - 1:
            end_idx = n
        else:
            end_idx = start_idx + group_size
        # 计算子组的平均值并添加到列表中
        group_mean = np.mean(data[start_idx:end_idx])
        group_means.append(group_mean)
    
    # 计算所有子组平均值的标准差，并除以 sqrt(groups) 来计算标准误差
    se = np.std(group_means, ddof=1) / np.sqrt(groups)
    
    return se

mean_angle = []
SE = []

# 加载数据的部分省略，假设 min_angle_results0 到 min_angle_results6 已经加载

min_angle_results = [min_angle_results0, min_angle_results1, min_angle_results2,
                     min_angle_results3, min_angle_results4, min_angle_results5,
                     min_angle_results6]

for data in min_angle_results:
    mean_angle.append(np.mean(data))
    SE.append(calculate_SE(data, 10))  # 使用刚才定义的函数并设置groups为10

parameter = [0, 0.3, 0.5, 0.6, 0.7, 0.9, 1]

data_to_save = np.column_stack((parameter, mean_angle, SE))
filename = "mean_angle.dat"
np.savetxt(filename, data_to_save, delimiter=' ', fmt=['%s', '%.2f', '%.2f'], header="Parameter Mean_Angle SE", comments='')

print(f'x={parameter}')
print(f'y={mean_angle}')
print(f'error={SE}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection

x = [0, 0.3, 0.5, 0.6, 0.7, 0.9, 1]
y = [149.53121067031464, 148.11988034001152, 147.91672201558782, 146.6890978439886, 139.18133634035692, 116.19585781335716, 101.00133111562592]
error = [0.10568883276347964, 0.575803116303573, 0.6345767972134332, 1.3492970160842714, 1.1098453157000643, 3.388457397157984, 6.091173135161627]

plt.rcParams.update({
    'font.family': 'Arial',  # 设置字体
    'axes.titlesize': 36,  # 标题字体大小
    'axes.labelsize': 36,  # 坐标轴标签字体大小
    'xtick.labelsize': 36,  # X轴刻度字体大小
    'ytick.labelsize': 36,  # Y轴刻度字体大小
    'legend.fontsize': 36,  # 图例字体大小
    'figure.figsize': (9.5, 6),  # 增加图像宽度，为color bar留出空间
    'axes.linewidth': 4,  # 坐标轴线宽
    'xtick.major.size': 10, # x 轴主刻度长度
    'ytick.major.size': 10, # y 轴主刻度长度
    'xtick.minor.size': 5, # x 轴次刻度长度
    'ytick.minor.size': 5, # y 轴主刻度长度
    'xtick.major.width': 4, # x 轴主刻度线宽
    'ytick.major.width': 4, # y 轴主刻度线宽
    'xtick.minor.width': 4, # x 轴次刻度线宽
    'ytick.minor.width': 4, # y 轴主刻度线宽
    'legend.fontsize': 28,    # 图例字体大小
    'axes.grid': False,  # 关闭背景网格
})

# 使用 color map 将颜色映射到数据点
cmap = plt.colormaps['plasma'] #coolwarm,inferno,magma,plasma
norm = mcolors.Normalize(vmin=min(x), vmax=max(x))
colors = [cmap(norm(value)) for value in x]

fig, ax = plt.subplots()  # 创建一个新的图形和轴

# 创建用于渐变线条的段
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# 创建一个LineCollection对象
lc = LineCollection(segments, colors=colors[:-1], linewidths=4)

# 添加到轴中
ax.add_collection(lc)

# 绘制带有标准误的点
for i in range(len(x)):
    ax.errorbar(x[i], y[i], yerr=error[i], fmt='o', 
                linewidth=4, capsize=16, capthick=20, ecolor=colors[i], 
                color=colors[i], marker='o', markersize=20, 
                markerfacecolor=colors[i], markeredgewidth=4, markeredgecolor='black')

ax.axhline(144.97, color='#E90F44', linestyle='dashed', linewidth=3)
ax.text(0.0, 138, 'PDB reference', color='#E90F44', fontsize=36)

# 添加图表标题和轴标签
ax.set_xlabel(r'$\epsilon_\text{HD}$')
ax.set_ylabel(r'$\theta$ (°)')

# 设置x轴的刻度为具体的组名
ax.set_xticks(np.arange(0, 1.1, 0.3))
ax.set_yticks(np.arange(90, 160, 20))

# x 轴刻度朝内
ax.tick_params(axis='x', which='both', direction='in')
# y 轴刻度朝内
ax.tick_params(axis='y', which='both', direction='in')

# 添加颜色条
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # 不设置数据
cbar = fig.colorbar(sm, ax=ax, ticks=[0, 0.5, 1])  # 添加颜色条和刻度
cbar.set_label(r'$\epsilon_\text{HD}$', rotation=270, labelpad=40, fontsize=36)  # 设置标签并向右旋转
cbar.ax.tick_params(labelsize=36)  # 设置颜色条刻度字体大小
cbar.outline.set_linewidth(4)  # 设置颜色条边框宽度

plt.tight_layout()  # 使布局更紧凑
plt.savefig("DNA_angle.png", dpi=300, bbox_inches='tight')
plt.show()